In [2]:
import numpy as np
import sympy as sp

In [3]:
labels = {
    'A': ("000", "1"),
    'B': ("001", "1"),
    'C': ("100", "1"),
    'D': ("101", "1"),
    'E': ("010", "0"),
    'F': ("011", "0"),
    'G': ("110", "0"),
    'H': ("111", "0")
}

In [4]:
# test
labels['A']

('000', '1')

In [5]:
# function to check if mask with ith cell dotted matches jth cell of sub
def doesMatch(mask, i, sub, j):
    m = len(mask)
    s = len(sub)
    if (i < 0 or j < 0 or j < i or j - i + m > s or i >= m or j >= s):
        # print(mask, ": ", i, j, m, s, "#", i < 0, j < 0, j < i, j - i + m >= s, i >= m, j >= s)
        # print(sub)
        return 0
    else:
        if (mask == sub[j - i: j - i + m]):
            return 1
        else:
            return 0


In [6]:
# test
doesMatch("01", 0, "010100011", 6)

1

In [7]:
# function to decide if a cell is able to switch its strategy regarding transition rules
def canCellFlip(left, center, right, transition_rules):
    n = transition_rules.shape[0]
    for i in range(n):
        if (transition_rules[i][0][0] == left and transition_rules[i][0][1] == center and transition_rules[i][0][2] == right and transition_rules[i][1][0] != center):
            return 1
    return 0

In [8]:
m = "0"
m[2 % len(m)]
#print(canCellFlip(m[-2], m[0], m[0], np.array([labels['E'], labels['F']])))

'0'

In [9]:
'''
def calcProb(u, i, v, j, transition_rules):
    r, p = sp.symbols("r p")
    len_u = len(u)
    len_v = len(v)
    


# consider at step t, a cell matches mask current_pattern. function to calculate the probability of
# the cell matches all masks patterns at the step t+1

def getProbMatchingNewMask(current_pattern, patterns, transition_rules):
    number_of_patterns = patterns.shape[0]
    probs = [0] * number_of_patterns
    r, p = sp.symbols("r p")
    for i in range(number_of_patterns):
        
    return np.array(probs)
'''

'\ndef calcProb(u, i, v, j, transition_rules):\n    r, p = sp.symbols("r p")\n    len_u = len(u)\n    len_v = len(v)\n    \n\n\n# consider at step t, a cell matches mask current_pattern. function to calculate the probability of\n# the cell matches all masks patterns at the step t+1\n\ndef getProbMatchingNewMask(current_pattern, patterns, transition_rules):\n    number_of_patterns = patterns.shape[0]\n    probs = [0] * number_of_patterns\n    r, p = sp.symbols("r p")\n    for i in range(number_of_patterns):\n        \n    return np.array(probs)\n'

In [10]:
# general function to calculate probability of conversion of sub-config A to B regarding certain transition rules
def generalCalcProb(A, B, transition_rules = np.array([labels['A'], labels['B'], labels['F'], labels['C'], labels['G'], labels['H']])):
    a = len(A)
    b = len(B)
    if (a != b or A[0] != B[0] or A[a - 1] != B[b - 1]):
        return 0
    r, p = sp.symbols("r p")
    r = 1
    for i in range(1, a - 1):    # ignoring first and last cell
        if (A[i] == B[i]):    # must remain on its strategy
            if (canCellFlip(A[i - 1], A[i], A[i + 1], transition_rules)):    # checking if the ith cell can flip
                r *= (1 - p)
            else:
                r *= 1
        else:
            if (canCellFlip(A[i - 1], A[i], A[i + 1], transition_rules)):
                r *= p
            else:
                return 0
    return r


In [11]:
# test
print(generalCalcProb("011110", "010110"))

p*(1 - p)**3


In [12]:
def generateAllBinaryStrings(length):
    arr = [""] * 2 ** length
    form = '#0' + str(length + 2) + 'b'
    for i in range(2 ** length):
        arr[i] = format(i, form)[2:]
    return np.array(arr)


In [13]:
# test
print(generateAllBinaryStrings(3))

['000' '001' '010' '011' '100' '101' '110' '111']


In [14]:
# function to calculate the expected difference of number of a list of patterns in a sub-config
def calcExpectedDifference(patterns, sub_config, dot_index, transition_rules = np.array([labels['A'], labels['B'], labels['F'], labels['C'], labels['G'], labels['H']])):
    # patterns_max_length = np.max([len(i) for i in patterns[:, 0]])
    k = len(sub_config) - 2
    n = 2 ** k
    binary_strings = generateAllBinaryStrings(k)
    number_of_patterns = patterns.shape[0]
    expected_variation = [0] * number_of_patterns
    for j in range(number_of_patterns):
        #
        total_expectation = 0
        current_number_of_occurrence = doesMatch(patterns[j][0], int(patterns[j][1]), sub_config[1:-1], dot_index - 1)
        for i in range(n):
            total_expectation += generalCalcProb(sub_config, sub_config[0] + binary_strings[i] + sub_config[-1], transition_rules) * (doesMatch(patterns[j][0], int(patterns[j][1]), binary_strings[i], dot_index - 1) - current_number_of_occurrence)
        #
        expected_variation[j] = total_expectation
    return np.array(expected_variation)


In [15]:
# test
pats = np.array([["001", 1]])
calcExpectedDifference(pats, "00011", 2)

array([-p**3 - 3*p**2*(1 - p) - 3*p*(1 - p)**2], dtype=object)

In [16]:
# test
pats = np.array([["0", 0], ["11", 1], ["01", 1]])
tr = np.array([labels['E'], labels['F']])
subs = generateAllBinaryStrings(4)
for i in subs:
    x = calcExpectedDifference(pats, i, 2, tr)
    print(i, x)

0000 [0 0 0]
0001 [0 0 0]
0010 [p 0 -p]
0011 [p 0 -p]
0100 [0 0 0]
0101 [0 0 0]
0110 [0 -p p]
0111 [0 -p p]
1000 [0 0 0]
1001 [0 0 0]
1010 [p 0 -p]
1011 [p 0 -p]
1100 [0 0 0]
1101 [0 0 0]
1110 [0 0 0]
1111 [0 0 0]


In [17]:
# test
pats = np.array([
    ["0010", 2],
    ["0011", 2],
    ["0101", 3],
    ["1101", 3],
    ["11", 1],
    ["00", 0],
    ["0010", 1],
    ["0011", 1],
    ["0101", 2],
    ["1101", 2]
])
dind = 2
tr = np.array([labels['B'], labels['E'], labels['F']])
subs = generateAllBinaryStrings(6)
for i in subs:
    x = calcExpectedDifference(pats, i, dind, tr)
    print(i, x)

000000 [0 0 0 0 0 0 0 0 0 0]
000001 [0 0 0 0 0 0 0 0 0 0]
000010 [0 0 0 0 0 -p**2 - p*(1 - p) p**2 p*(1 - p) 0 0]
000011 [0 0 0 0 0 -p**2 - p*(1 - p) p**2 p*(1 - p) 0 0]
000100 [0 0 0 0 0 p*(1 - p) -p**2 - 2*p*(1 - p) 0 0 0]
000101 [0 0 0 0 0 p*(1 - p) -p**2 - 2*p*(1 - p) 0 0 0]
000110 [0 0 0 0 0 p*(1 - p) 0 -p**2 - 2*p*(1 - p) 0 0]
000111 [0 0 0 0 0 p*(1 - p) 0 -p**2 - 2*p*(1 - p) 0 0]
001000 [0 0 0 0 p*(1 - p) p**2 + p*(1 - p) 0 0 0 0]
001001 [0 0 0 0 p**2*(1 - p) + p*(1 - p)**2 p**3 + 2*p**2*(1 - p) + p*(1 - p)**2
 0 0 0 0]
001010 [0 0 0 0 p**2*(1 - p) + p*(1 - p)**2 p**3 + 2*p**2*(1 - p) + p*(1 - p)**2
 0 0 0 0]
001011 [0 0 0 0 p**2*(1 - p) + p*(1 - p)**2 p**3 + 2*p**2*(1 - p) + p*(1 - p)**2
 0 0 0 0]
001100 [0 0 0 0 p*(1 - p) 0 p*(1 - p) 0 0 0]
001101 [0 0 0 0 p*(1 - p) 0 p*(1 - p) 0 0 0]
001110 [0 0 0 0 p*(1 - p) 0 0 p*(1 - p) 0 0]
001111 [0 0 0 0 p*(1 - p) 0 0 p*(1 - p) 0 0]
010000 [0 0 0 0 0 0 0 0 0 0]
010001 [0 0 0 0 0 0 0 0 0 0]
010010 [0 0 0 0 0 -p**3 - 2*p**2*(1 - p) - p*(1

In [28]:
# test
pats = np.array([
    ["0010", 2],
    ["0011", 2],
    ["0101", 3],
    ["1101", 3],
    ["11", 1],
    ["00", 0],
    ["0010", 1],
    ["0011", 1],
    ["0101", 2],
    ["1101", 2]

    # ["1", 0],
    # ["011", 2],
    # ["101", 2]
])
dind = 2
tr = np.array([labels['B'], labels['E'], labels['F']])
subs = generateAllBinaryStrings(6)
for i in subs:
    x = calcExpectedDifference(pats, i, dind, tr)
    print(i, x)

000000 [0 0 0 0 0 0 0 0 0 0]
000001 [0 0 0 0 0 0 0 0 0 0]
000010 [0 0 0 0 0 -p**2 - p*(1 - p) p**2 p*(1 - p) 0 0]
000011 [0 0 0 0 0 -p**2 - p*(1 - p) p**2 p*(1 - p) 0 0]
000100 [0 0 0 0 0 p*(1 - p) -p**2 - 2*p*(1 - p) 0 0 0]
000101 [0 0 0 0 0 p*(1 - p) -p**2 - 2*p*(1 - p) 0 0 0]
000110 [0 0 0 0 0 p*(1 - p) 0 -p**2 - 2*p*(1 - p) 0 0]
000111 [0 0 0 0 0 p*(1 - p) 0 -p**2 - 2*p*(1 - p) 0 0]
001000 [0 0 0 0 p*(1 - p) p**2 + p*(1 - p) 0 0 0 0]
001001 [0 0 0 0 p**2*(1 - p) + p*(1 - p)**2 p**3 + 2*p**2*(1 - p) + p*(1 - p)**2
 0 0 0 0]
001010 [0 0 0 0 p**2*(1 - p) + p*(1 - p)**2 p**3 + 2*p**2*(1 - p) + p*(1 - p)**2
 0 0 0 0]
001011 [0 0 0 0 p**2*(1 - p) + p*(1 - p)**2 p**3 + 2*p**2*(1 - p) + p*(1 - p)**2
 0 0 0 0]
001100 [0 0 0 0 p*(1 - p) 0 p*(1 - p) 0 0 0]
001101 [0 0 0 0 p*(1 - p) 0 p*(1 - p) 0 0 0]
001110 [0 0 0 0 p*(1 - p) 0 0 p*(1 - p) 0 0]
001111 [0 0 0 0 p*(1 - p) 0 0 p*(1 - p) 0 0]
010000 [0 0 0 0 0 0 0 0 0 0]
010001 [0 0 0 0 0 0 0 0 0 0]
010010 [0 0 0 0 0 -p**3 - 2*p**2*(1 - p) - p*(1

# final output

In [18]:
#
def getAllDifferencesOnExpectationWithGivenPatterns(patterns, dot_index, transition_rules = np.array([labels['A'], labels['B'], labels['F'], labels['C'], labels['G'], labels['H']])):
    pattern_max_length = np.max([len(i) for i in patterns[:, 0]])
    m = patterns.shape[0]

    # binary sub-config length should be equal with pattern_length + 2
    k = pattern_max_length + 2
    n = 2 ** k
    binary_sub_configs = generateAllBinaryStrings(k)
    diffs = [ [0] * m for i in range(n)]
    for j in range(n):
        expectations_of_current_sub_config = calcExpectedDifference(patterns, binary_sub_configs[j], dot_index, transition_rules)
        diffs[j] = expectations_of_current_sub_config
    
    return np.array(diffs).T


### EF and EFG 🟢

In [19]:
####################### test 1
pats = np.array([["0", 0], ["11", 1], ["01", 1]])
dind = 2
tr = np.array([labels['E'], labels['F']])
weights = np.array([0, 1, 1])

x = getAllDifferencesOnExpectationWithGivenPatterns(pats, dind, tr)
y = np.dot(weights, x)
for i in y:
    print(i)

0
0
-p
-p
0
0
0
0
0
0
-p
-p
0
0
0
0


### BEF 🔴

In [26]:
####################### test 2
pats = np.array([
    ["0010", 2],
    ["0011", 2],
    ["0101", 3],
    ["1101", 3],
    ["11", 1],
    ["00", 0],
    ["0010", 1],
    ["0011", 1],
    ["0101", 2],
    ["1101", 2]

    # ["1", 0],
    # ["011", 2],
    # ["101", 2]
])
dind = 2
tr = np.array([labels['B'], labels['E'], labels['F']])
p = sp.symbols("p")
c = -sp.floor(3 / p) - 1
b = -1
a = -2 * c + 2
weights = np.array([
    a,
    a + b,
    a + b,
    a + b,
    a,
    0,
    0,
    0,
    0,
    c

    # a,
    # b,
    # c
])

x = getAllDifferencesOnExpectationWithGivenPatterns(pats, dind, tr)
y = np.dot(weights, x)
for i in y:
    print(i)

0
0
0
0
0
0
0
0
p*(1 - p)*(2*floor(3/p) + 4)
(p**2*(1 - p) + p*(1 - p)**2)*(2*floor(3/p) + 4)
(p**2*(1 - p) + p*(1 - p)**2)*(2*floor(3/p) + 4)
(p**2*(1 - p) + p*(1 - p)**2)*(2*floor(3/p) + 4)
p*(1 - p)*(2*floor(3/p) + 4)
p*(1 - p)*(2*floor(3/p) + 4)
p*(1 - p)*(2*floor(3/p) + 4)
p*(1 - p)*(2*floor(3/p) + 4)
0
0
0
0
0
0
0
0
-p*(2*floor(3/p) + 4)
(-p**2 - p*(1 - p))*(2*floor(3/p) + 4)
(-p**2 - p*(1 - p))*(2*floor(3/p) + 4)
(-p**2 - p*(1 - p))*(2*floor(3/p) + 4)
-p*(2*floor(3/p) + 4)
-p*(2*floor(3/p) + 4)
-p*(2*floor(3/p) + 4)
-p*(2*floor(3/p) + 4)
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


### B and BC 🟢

In [21]:
####################### test 3
pats = np.array([
    ["00", 0],
    ["001", 1],
    ["101", 1],
    ["1", 0]
])
dind = 2
tr = np.array([labels['B'], labels['C']])
weights = np.array([
    1,
    1,
    0,
    0
])

x = getAllDifferencesOnExpectationWithGivenPatterns(pats, dind, tr)
y = np.dot(weights, x)
for i in y:
    print(i)

0
0
-p
-p
0
0
0
0
-p
-p**2 - 2*p*(1 - p)
0
0
0
0
0
0
0
-p**2
-p**2 - 2*p*(1 - p)
-p**2 - 2*p*(1 - p)
0
0
0
0
-p
-p**2 - 2*p*(1 - p)
0
0
0
0
0
0


### BDE and BCDE 🔴

In [22]:
####################### test 3
pats = np.array([
    # ["011", 1],
    # ["010", 1],
    # ["11", 1],
    # ["00", 0],
    # ["011", 0],
    # ["010", 0]
    ["0", 0]
])
dind = 1
tr = np.array([labels['B'], labels['D'], labels['E']])
weights = np.array([
    1
])

x = getAllDifferencesOnExpectationWithGivenPatterns(pats, dind, tr)
y = np.dot(weights, x)
for i in y:
    print(i)

0
-p
p
0
0
-p
0
0


### BE and BCE 🟢

In [23]:
####################### test 4
pats = np.array([
    ["0", 0],
    ["010", 1],
    ["01011", 3]
])
dind = 2
tr = np.array([labels['B'], labels['D'], labels['E']])
weights = np.array([
    3,
    3,
    1
])

x = getAllDifferencesOnExpectationWithGivenPatterns(pats, dind, tr)
y = np.dot(weights, x)
for i in y:
    print(i)

0
0
0
0
0
0
0
0
-3*p*(1 - p)
-3*p**2*(1 - p) - 3*p*(1 - p)**2
-3*p**3*(1 - p) - 6*p**2*(1 - p)**2 - 3*p*(1 - p)**3
-3*p**2*(1 - p) - 3*p*(1 - p)**2
-3*p
-3*p**2 - 3*p*(1 - p)
-3*p
-3*p
-3*p*(1 - p)
-3*p**2*(1 - p) - 3*p*(1 - p)**2
-3*p**3*(1 - p) - 6*p**2*(1 - p)**2 - 3*p*(1 - p)**3
-3*p**2*(1 - p) - 3*p*(1 - p)**2
-3*p**3*(1 - p) - 9*p**2*(1 - p)**2 - 6*p*(1 - p)**3
-3*p**4*(1 - p) - 12*p**3*(1 - p)**2 - 15*p**2*(1 - p)**3 - 6*p*(1 - p)**4
-3*p**2*(1 - p) - 6*p*(1 - p)**2
-3*p**2*(1 - p) - 6*p*(1 - p)**2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-6*p**2*(1 - p) - 3*p*(1 - p)**2
-6*p**3*(1 - p) - 9*p**2*(1 - p)**2 - 3*p*(1 - p)**3
-6*p**4*(1 - p) - 15*p**3*(1 - p)**2 - 12*p**2*(1 - p)**3 - 3*p*(1 - p)**4
-6*p**3*(1 - p) - 9*p**2*(1 - p)**2 - 3*p*(1 - p)**3
-3*p**2 - 3*p*(1 - p)
-3*p**3 - 6*p**2*(1 - p) - 3*p*(1 - p)**2
-3*p**2 - 3*p*(1 - p)
-3*p**2 - 3*p*(1 - p)
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-3*p*(1 - p)
-3*p**2*(1 - p) - 3*p*(1 - p)**2
-3*p**3*(1 - p) - 6*p**2*(1 - p)**2 - 3*p

### BEFG

In [24]:
####################### test 5
pats = np.array([
    ["011", 1],
    ["11", 1],
    ["010", 1]

    # ["1", 0],
    # ["010", 1]

])
dind = 2
tr = np.array([labels['B'], labels['E'], labels['F'], labels['G']])
weights = np.array([
    5,
    5,
    6

    # 5,
    # 1
])

x = getAllDifferencesOnExpectationWithGivenPatterns(pats, dind, tr)
y = np.dot(weights, x)
for i in y:
    print(i)

0
0
6*p**2 + 5*p*(1 - p)
6*p**2 + 5*p*(1 - p)
-6*p**2 - 7*p*(1 - p)
-6*p**2 - 7*p*(1 - p)
-5*p**3 - 10*p**2*(1 - p) - 4*p*(1 - p)**2
-5*p**2 - 5*p*(1 - p)
0
0
0
0
-5*p**2 - 4*p*(1 - p)
-5*p**2 - 4*p*(1 - p)
p**2
0
0
0
6*p**2 + 5*p*(1 - p)
6*p**2 + 5*p*(1 - p)
-6*p
-6*p
-5*p**2 - 4*p*(1 - p)
-5*p
0
0
0
0
-5*p
-5*p
0
0


### BCDEF

In [25]:
####################### test 5
pats = np.array([
    ["00", 0],
    ["01", 0],
    ["1", 0]
])
dind = 1
tr = np.array([labels['B'], labels['C'], labels['D'], labels['E'], labels['F']])
weights = np.array([
    1,
    2,
    0
])

x = getAllDifferencesOnExpectationWithGivenPatterns(pats, dind, tr)
y = np.dot(weights, x)
for i in y:
    print(i)

0
p
-2*p**2 - 3*p*(1 - p)
-2*p**2 - 3*p*(1 - p)
2*p**2 + p*(1 - p)
2*p**2 + p*(1 - p)
2*p
2*p
-p
-p**2
-2*p**2 - 3*p*(1 - p)
-2*p**2 - 3*p*(1 - p)
0
0
0
0
